In [ ]:
import cv2 
import pandas as pd
import numpy as np

In [ ]:
class FaceToDF:
    def __init__(self):
        self.detector = cv2.FaceDetectorYN.create(
            model='C:/Users/Stepan/Documents/ClassMood/yunet_headpose/face_detection_yunet_2023mar.onnx',
            config='',
            input_size=(67, 69),
            score_threshold=0.8,
            nms_threshold=0.3,
            top_k=5000
        )
    
        if cv2.cuda.getCudaEnabledDeviceCount() > 0:
            self.detector.setBackend(cv2.dnn.DNN_BACKEND_CUDA)
            self.detector.setTarget(cv2.dnn.DNN_TARGET_CUDA)
            print('Используется GPU')
        else:
            print('Используется CPU')

    def head_pose_img(self, path):
        import math

        image = cv2.imread(path)
        self.detector.setInputSize(image.shape[1::-1])
        faces = self.detector.detect(image)[1]
        
        if faces is None:
            return pd.DataFrame()
        
        columns = [
            'face_id', 'x', 'y', 'width', 'height',
            'le_x', 'le_y',
            're_x', 're_y',
            'n_x', 'n_y',
            'ml_x', 'ml_y',
            'mr_x', 'mr_y',
            'confidence',
            'tilt_x_raw', 'tilt_y_raw'
        ]
        data = []

        for i, face in enumerate(faces):
            row = [i + 1] + face.tolist() + [0.0, 0.0]
            data.append(row)
        
        df = pd.DataFrame(data, columns=columns)
        for index, row in df.iterrows():
            dif_x = ((row['le_x'] + row['re_x'] + row['ml_x'] + row['mr_x']) / 4) - row['n_x']
            df.loc[index, 'tilt_x_raw'] = round(dif_x / df['width'][0], 6)

            dif_y = ((row['le_y'] + row['re_y'] + row['ml_y'] + row['mr_y']) / 4) - row['n_y']
            df.loc[index, 'tilt_y_raw'] = round(dif_y / df['height'][0], 6)
        
        return faces

In [ ]:
face_to_dataframe = FaceToDF()
face_to_dataframe.head_pose_img('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/test_img.jpg')

внизу приведен отладчик, строит на одном лице на фото 5 ключевых точек с подписями

In [ ]:
def draw_landmarks_on_image(image, landmarks, labels=None, color=(0, 0, 255), radius=5, thickness=-1, font_scale=1, font_thickness=2):
    output_image = image.copy()

    for i, (x, y) in enumerate(landmarks):
        cx, cy = int(x), int(y)
        
        cv2.circle(output_image, (cx, cy), radius, color, thickness)
        
        if labels and i < len(labels):
            cv2.putText(
                output_image,
                str(labels[i]),
                (cx + 5, cy - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale,
                (255, 0, 0),
                font_thickness
            )

    return output_image


def otladka(imgpath):
    landmarks_flat = face_to_dataframe.head_pose_img(imgpath)[0][4:-1]
    landmarks = landmarks_flat.reshape(-1, 2)

    image_with_landmarks = draw_landmarks_on_image(
        image=cv2.imread(imgpath),
        landmarks=landmarks,
        labels=['LE', 'RE', 'N', 'LM', 'RM'],
        color=(0, 0, 255),
        radius=5
    )


    output_path = imgpath
    cv2.imwrite(output_path, image_with_landmarks)

otladka('C:/Users/Stepan/Documents/ClassMood/yunet_headpose/test_img21.jpg')